In [10]:
import os
import pandas as pd
import numpy as np
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical, Sequence
from sklearn.model_selection import train_test_split
import cv2

In [11]:
pd.set_option('display.max_colwidth', None)


In [ ]:

full_training = []

paths = [
    ('C:/Users/savin/Downloads/FracAtlas/FracAtlas/images/Fractured', 0),
    ('C:/Users/savin/Downloads/FracAtlas/FracAtlas/images/Non_fractured', 1)
]

for folder, label in paths:
    for x in os.listdir(folder):
        if x.lower().endswith(('.png', '.jpg', '.jpeg')):
            full_training.append({'Image_path': os.path.join(folder, x), 'Classification': label})

mura_csv = pd.read_csv('C:/Users/savin/Downloads/MURA-v1.1/train_image_paths.csv')
base_dir = 'C:/Users/savin/Downloads/'
for x in mura_csv.iloc[:, 0]:
    label = 0 if 'positive' in x else 1 if 'negative' in x else None
    if label is not None:
        full_training.append({'Image_path': os.path.join(base_dir, x), 'Classification': label})

extra_paths = [
    ('C:/Users/savin/Downloads/data/data/train/b', 0),
    ('C:/Users/savin/Downloads/data/data/train/nb', 1),
    ('C:/Users/savin/Downloads/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/fractured', 0),
    ('C:/Users/savin/Downloads/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train/not fractured', 1)

]

for folder, label in extra_paths:
    for x in os.listdir(folder):
        if x.lower().endswith(('.png', '.jpg', '.jpeg')):
            full_training.append({'Image_path': os.path.join(folder, x), 'Classification': label})


full_training = pd.DataFrame(full_training)


In [ ]:
IMG_SIZE = (128, 128)
images, labels = [], []

for _, row in full_training.iterrows():
    img = load_img(row['Image_path'], target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    images.append(img_array)

    labels.append(int(row['Classification']))

images = np.array(images, dtype="float32")
labels = np.array(labels, dtype="int")

labels_categorical = to_categorical(labels, num_classes=2)

X_train, X_val, y_train, y_val = train_test_split(
    images, labels_categorical, test_size=0.2, stratify=labels, random_state=42
)


In [ ]:
class NumpyDataGenerator(Sequence):
    def __init__(self, images, labels, batch_size=64, shuffle=True):
        self.images = images
        self.labels = labels

        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(images))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        
        return self.images[batch_indices], self.labels[batch_indices]

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu', name="target_conv_layer"),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_gen = NumpyDataGenerator(X_train, y_train, batch_size=64)
val_gen = NumpyDataGenerator(X_val, y_val, batch_size=64)


In [16]:
model.fit(train_gen, epochs=25, validation_data=val_gen)


Epoch 1/25
770/770 [==============================] - 12s 13ms/step - loss: 0.6001 - accuracy: 0.6640 - val_loss: 0.5387 - val_accuracy: 0.7109
Epoch 2/25
770/770 [==============================] - 10s 13ms/step - loss: 0.4905 - accuracy: 0.7427 - val_loss: 0.4944 - val_accuracy: 0.7357
Epoch 3/25
770/770 [==============================] - 9s 11ms/step - loss: 0.3864 - accuracy: 0.8111 - val_loss: 0.5023 - val_accuracy: 0.7531
Epoch 4/25
770/770 [==============================] - 9s 11ms/step - loss: 0.2620 - accuracy: 0.8824 - val_loss: 0.6239 - val_accuracy: 0.7386
Epoch 5/25
770/770 [==============================] - 9s 11ms/step - loss: 0.1542 - accuracy: 0.9368 - val_loss: 0.7781 - val_accuracy: 0.7470
Epoch 6/25
770/770 [==============================] - 9s 11ms/step - loss: 0.0874 - accuracy: 0.9677 - val_loss: 1.0455 - val_accuracy: 0.7454
Epoch 7/25
770/770 [==============================] - 9s 11ms/step - loss: 0.0574 - accuracy: 0.9814 - val_loss: 1.2577 - val_accuracy: 0.74

In [17]:
img_path = 'C:/Users/savin/GATE Project/download.jpg'
image = load_img(img_path, target_size=IMG_SIZE)
image_array = img_to_array(image) / 255.0
image_array = np.expand_dims(image_array, axis=0)

output = model.predict(image_array)
predicted_class_index = np.argmax(output)
class_labels = ['Fracture Detected', 'No Fracture was Detected']
print("Prediction:", class_labels[predicted_class_index])


1/1 [==============================] - 0s 121ms/step
Prediction: Fracture Detected


In [1]:
def get_grad_cam(model, image, layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], 
        [model.get_layer(layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(image)
        class_idx = tf.argmax(predictions[0])
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

heatmap = get_grad_cam(model, image_array, 'target_conv_layer')
heatmap_resized = cv2.resize(heatmap, (128, 128))
heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET)

original = cv2.imread(img_path)
original = cv2.resize(original, (128, 128))
superimposed = cv2.addWeighted(original, 0.6, heatmap_colored, 0.4, 0)

plt.imshow(cv2.cvtColor(superimposed, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Grad-CAM Highlight")
plt.show()


NameError: name 'model' is not defined

In [ ]:
model.save('my_model1.keras')
